# WeatherBench Datasets

There are many ways to acquire ERA5 datasets. 
While I personally prefer the CDSAPI, the server is extremely slow these days due to the update (updated: 2024-02-21).
However, the weatherbench2 dataset is available for all.

In [3]:
import numpy as np
import cdsapi
from pathlib import Path
import autoroot
import datetime
from earth2mip import inference_ensemble, registry
from earth2mip.networks import get_model
from earth2mip.initial_conditions import cds
from earth2mip.inference_ensemble import run_basic_inference
import matplotlib.pyplot as plt
from bayesevt._src.data.ics import LocalDataSource
import xarray as xr

%matplotlib inline
%load_ext autoreload
%autoreload 2

## ERA5 Data

The first dataset is the reanalysis data.

* Resolution: `0.25x0.25`
* Temporal Freq: `1h`
* Period: `1959-2023`
* 48 Variables

In [4]:
# cloud_dir = "/pool/proyectos/CLINT/sa4attrs/data/raw/1959-2023_01_10-full_37-1h-0p25deg-chunk-1.zarr"

In [5]:
# ds = xr.open_zarr(local_dir)
# ds

## ERA5 + Derived Variables

These have some extra variables that can be derived from the original datasets.

In [13]:
local_dir = "/pool/proyectos/CLINT/sa4attrs/data/raw/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr"

In [14]:
ds = xr.open_zarr(local_dir)
# ds

In [15]:
f"{ds.nbytes / (1024*1024*1024):_}"

'82_853.25300930068'

In [20]:
ds["temperature"]

<xarray.DataArray 'temperature' (time: 93544, level: 13, latitude: 721,
                                 longitude: 1440)>
dask.array<open_dataset-temperature, shape=(93544, 13, 721, 1440), dtype=float32, chunksize=(1, 13, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * level      (level) int64 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 1959-01-01 ... 2023-01-10T18:00:00
Attributes:
    long_name:      Temperature
    short_name:     t
    standard_name:  air_temperature
    units:          K

## Climatology



In [9]:
local_dir = "/pool/proyectos/CLINT/sa4attrs/data/raw/1990-2017_climatology_6h_1440x721.zarr"

In [10]:
ds = xr.open_zarr(local_dir)
ds

<xarray.Dataset>
Dimensions:                                         (hour: 4, dayofyear: 366,
                                                     latitude: 721,
                                                     longitude: 1440, level: 13)
Coordinates:
  * dayofyear                                       (dayofyear) int64 1 ... 366
  * hour                                            (hour) int64 0 6 12 18
  * latitude                                        (latitude) float32 90.0 ....
  * level                                           (level) int64 50 ... 1000
  * longitude                                       (longitude) float32 0.0 ....
Data variables: (12/53)
    10m_u_component_of_wind                         (hour, dayofyear, latitude, longitude) float32 dask.array<chunksize=(3, 3, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                         (hour, dayofyear, latitude, longitude) float32 dask.array<chunksize=(3, 3, 721, 1440), meta=np.ndarray>
    10m_wind_speed                                  (hour, dayofyear, latitude, longitude) float32 dask.array<chunksize=(3, 3, 721, 1440), meta=np.ndarray>
    2m_dewpoint_temperature                         (hour, dayofyear, latitude, longitude) float32 dask.array<chunksize=(3, 3, 721, 1440), meta=np.ndarray>
    2m_temperature                                  (hour, dayofyear, latitude, longitude) float32 dask.array<chunksize=(3, 3, 721, 1440), meta=np.ndarray>
    above_ground                                    (hour, dayofyear, level, latitude, longitude) float32 dask.array<chunksize=(3, 3, 1, 721, 1440), meta=np.ndarray>
    ...                                              ...
    volumetric_soil_water_layer_1                   (hour, dayofyear, latitude, longitude) float32 dask.array<chunksize=(3, 3, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_2                   (hour, dayofyear, latitude, longitude) float32 dask.array<chunksize=(3, 3, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_3                   (hour, dayofyear, latitude, longitude) float32 dask.array<chunksize=(3, 3, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_4                   (hour, dayofyear, latitude, longitude) float32 dask.array<chunksize=(3, 3, 721, 1440), meta=np.ndarray>
    vorticity                                       (hour, dayofyear, level, latitude, longitude) float32 dask.array<chunksize=(3, 3, 1, 721, 1440), meta=np.ndarray>
    wind_speed                                      (hour, dayofyear, level, latitude, longitude) float32 dask.array<chunksize=(3, 3, 1, 721, 1440), meta=np.ndarray>

In [11]:
f"{ds.nbytes / (1024*1024*1024):_}"

'1_319.3343796096742'

In [22]:
ds_ = xr.open_mfdataset("/pool/proyectos/CLINT/sa4attrs/data/raw/fcnv2sm*", engine="cfgrib")

ds_

<xarray.Dataset>
Dimensions:        (latitude: 721, longitude: 1440, isobaricInhPa: 13)
Coordinates:
    number         int64 0
    time           datetime64[ns] 2021-08-01
    step           timedelta64[ns] 00:00:00
    surface        float64 ...
  * latitude       (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude      (longitude) float64 -180.0 -179.8 -179.5 ... 179.5 179.8
    valid_time     datetime64[ns] 2021-08-01
  * isobaricInhPa  (isobaricInhPa) float64 1e+03 925.0 850.0 ... 100.0 50.0
Data variables: (12/13)
    sp             (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    tcwv           (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    msl            (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    u10            (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    v10            (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    t2m            (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    ...             ...
    v100           (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    z              (isobaricInhPa, latitude, longitude) float32 dask.array<chunksize=(13, 721, 1440), meta=np.ndarray>
    t              (isobaricInhPa, latitude, longitude) float32 dask.array<chunksize=(13, 721, 1440), meta=np.ndarray>
    u              (isobaricInhPa, latitude, longitude) float32 dask.array<chunksize=(13, 721, 1440), meta=np.ndarray>
    v              (isobaricInhPa, latitude, longitude) float32 dask.array<chunksize=(13, 721, 1440), meta=np.ndarray>
    r              (isobaricInhPa, latitude, longitude) float32 dask.array<chunksize=(13, 721, 1440), meta=np.ndarray>
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-02-21T02:00 GRIB to CDM+CF via cfgrib-0.9.1...